In [1]:
import pandas as pd

#download any month of data for yellow cabs https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
data = pd.read_parquet("/Users/sirsh/Downloads/yellow_tripdata_2023-01.parquet")
#get the zone lookup from some site 
zones = pd.read_csv("/Users/sirsh/Downloads/taxi+_zone_lookup.csv")
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [2]:
zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [3]:
columns = {
    "tpep_pickup_datetime": 'pick_up_at',
    "tpep_dropoff_datetime": 'drop_off_at',
    "passenger_count": "passenger_count",
    "trip_distance": "trip_distance",
    "payment_type": "payment_type",
    "congestion_surcharge": "congestion_surcharge",
    "airport_fee": "airport_fee",
    "Borough": "borough_pick_up",
    "Zone": "zone_pick_up",
    "Borough_drop_off": "borough_drop_off",
    "Zone_drop_off": "zone_drop_off"
}

def payment_types (i):
    """
    from the data dict for sourced data on site
    """
    return ['Credit card', 'Cash', 'No charge', 'Dispute', 'Unknown'][i-1]

data = pd.merge(data,zones, left_on='PULocationID', right_on='LocationID', suffixes=['', '_pick_up'])
data = pd.merge(data,zones, left_on='DOLocationID', right_on='LocationID', suffixes=['', '_drop_off'])
data['payment_type'] = data['payment_type'].map(payment_types)
data = data.rename(columns=columns)
data = data.drop(columns=[c for c in data.columns if c not in columns.values()],index=1).reset_index()
sample = data[::100].reset_index(drop=True)
sample['pick_up_at'] = pd.to_datetime(sample['pick_up_at'])
sample['drop_off_at'] = pd.to_datetime(sample['drop_off_at'])
sample.head()

,index,pick_up_at,drop_off_at,passenger_count,trip_distance,payment_type,congestion_surcharge,airport_fee,borough_pick_up,zone_pick_up,borough_drop_off,zone_drop_off
0,0,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,Cash,2.5,0.0,Manhattan,Midtown Center,Manhattan,Lenox Hill West
1,101,2023-01-02 16:27:22,2023-01-02 16:37:07,0.0,1.70,Credit card,2.5,0.0,Manhattan,Midtown Center,Manhattan,Lenox Hill West
2,201,2023-01-03 17:20:41,2023-01-03 17:30:33,1.0,1.44,Cash,2.5,0.0,Manhattan,Midtown Center,Manhattan,Lenox Hill West
3,301,2023-01-04 13:45:55,2023-01-04 13:54:53,1.0,1.09,Credit card,2.5,0.0,Manhattan,Midtown Center,Manhattan,Lenox Hill West
4,401,2023-01-04 21:24:06,2023-01-04 21:31:52,1.0,1.81,Cash,2.5,0.0,Manhattan,Midtown Center,Manhattan,Lenox Hill West


In [4]:
# from tqdm import tqdm
# import time
# from langchain.utilities import WikipediaAPIWrapper
# wikipedia = WikipediaAPIWrapper()
# trivia = []
 

# for zone in tqdm(sample['zone_pick_up'].unique()):
#     try:
#         trivia.append(   {"entity_type" : 'nyc_zone', "entity_key": zone, 'text' : wikipedia.run(zone)})
#         time.sleep(2)
#     except:
#         pass
        
# trivia = pd.DataFrame(trivia).reset_index()
# trivia['id'] = trivia['index']
# trivia.to_csv("/Users/sirsh/Downloads/nyc_zones.csv", index=None)
# trivia  


In [18]:
passengers = pd.read_csv("/Users/sirsh/Downloads/avengers.csv", encoding='latin-1')
passengers = passengers[['URL', 'Name/Alias', 'Appearances', 'Gender']].rename(columns={'Name/Alias':'Name', 'URL' : 'uri'})
passengers.columns = [c.lower() for c in passengers.columns]
passengers

,uri,name,appearances,gender
0,http://marvel.wikia.com/Henry_Pym_(Earth-616),"Henry Jonathan ""Hank"" Pym",1269,MALE
1,http://marvel.wikia.com/Janet_van_Dyne_(Earth-...,Janet van Dyne,1165,FEMALE
2,http://marvel.wikia.com/Anthony_Stark_(Earth-616),"Anthony Edward ""Tony"" Stark",3068,MALE
3,http://marvel.wikia.com/Robert_Bruce_Banner_(E...,Robert Bruce Banner,2089,MALE
4,http://marvel.wikia.com/Thor_Odinson_(Earth-616),Thor Odinson,2402,MALE
...,...,...,...,...
168,http://marvel.wikia.com/Eric_Brooks_(Earth-616)#,Eric Brooks,198,MALE
169,http://marvel.wikia.com/Adam_Brashear_(Earth-6...,Adam Brashear,29,MALE
170,http://marvel.wikia.com/Victor_Alvarez_(Earth-...,Victor Alvarez,45,MALE
171,http://marvel.wikia.com/Ava_Ayala_(Earth-616)#,Ava Ayala,49,FEMALE


In [21]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
def scrape_html_paragraphs(uri):
    """
    util - we dont care about errors - this is a try or ignore for test data
    """
    try:
        response = requests.get(uri)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            title = soup.title.string
            paragraphs = soup.find_all('p')
            text_paragraphs = [p.get_text() for p in paragraphs]
            cleaned_text = '\n'.join(text_paragraphs)
            return cleaned_text
        else:
            return None
    except Exception as e:
        return None

bios = []
for record in tqdm(passengers.to_dict('records')):  
    bios.append(   {"entity_type" : 'people', "entity_key": record['name'], 'text' : scrape_html_paragraphs(record['uri'])})
    
bios = pd.DataFrame(bios)
bios.to_csv("/Users/sirsh/Downloads/marvel_bios.csv", index=None)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [01:21<00:00,  2.11it/s]


In [6]:
import numpy as np
sample['passenger_name'] = np.random.choice(passengers['name'], len(sample))
sample

,index,pick_up_at,drop_off_at,passenger_count,trip_distance,payment_type,congestion_surcharge,airport_fee,borough_pick_up,zone_pick_up,borough_drop_off,zone_drop_off,passenger_name
0,0,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,Cash,2.5,0.00,Manhattan,Midtown Center,Manhattan,Lenox Hill West,Phillip Javert
1,101,2023-01-02 16:27:22,2023-01-02 16:37:07,0.0,1.70,Credit card,2.5,0.00,Manhattan,Midtown Center,Manhattan,Lenox Hill West,Lyra
2,201,2023-01-03 17:20:41,2023-01-03 17:30:33,1.0,1.44,Cash,2.5,0.00,Manhattan,Midtown Center,Manhattan,Lenox Hill West,Sharon Carter
3,301,2023-01-04 13:45:55,2023-01-04 13:54:53,1.0,1.09,Credit card,2.5,0.00,Manhattan,Midtown Center,Manhattan,Lenox Hill West,Dennis Sykes
4,401,2023-01-04 21:24:06,2023-01-04 21:31:52,1.0,1.81,Cash,2.5,0.00,Manhattan,Midtown Center,Manhattan,Lenox Hill West,Sam Alexander
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30663,3066301,2023-01-07 17:42:53,2023-01-07 18:19:14,1.0,0.00,Credit card,0.0,0.00,Manhattan,Central Harlem North,Queens,Hollis,Christopher Powell
30664,3066401,2023-01-14 05:05:48,2023-01-14 05:05:53,1.0,0.07,Cash,0.0,0.00,Staten Island,Stapleton,Staten Island,Grymes Hill/Clifton,Sam Alexander
30665,3066501,2023-01-20 20:07:50,2023-01-20 21:14:32,1.0,29.10,Credit card,0.0,1.25,Queens,JFK Airport,Staten Island,Mariners Harbor,Craig Hollis
30666,3066601,2023-01-20 15:11:58,2023-01-20 16:15:39,1.0,26.50,Credit card,0.0,1.25,Queens,JFK Airport,Staten Island,Westerleigh,Brunnhilde


In [8]:
from monologue.entities.examples import NycTripEvent
NycTripEvent(**sample.iloc[0])

```json{"passenger_name": "Phillip Javert", "pick_up_at": "2023-01-01 00:32:10", "drop_off_at": "2023-01-01 00:40:36", "passenger_count": 1, "trip_distance": 0.97, "payment_type": "Cash", "congestion_surcharge": 2.5, "airport_fee": 0.0, "borough_pick_up": "Manhattan", "zone_pick_up": "Midtown Center", "borough_drop_off": "Manhattan", "zone_drop_off": "Lenox Hill West", "__type__": "NycTripEvent", "__key__": "passenger_name", "__namespace__": "examples"}```